# **VISCOSITY SOLUTIONS THROUGH PENALTY METHODS**

## **1. CODE**

### Firedrake

In [3]:
try:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-release-real.sh" -O "/tmp/firedrake-install.sh"
    !bash "/tmp/firedrake-install.sh"
    from firedrake import *  # noqa: F401
except:
    from firedrake import *  # noqa: F401

--2025-12-16 16:26:37--  https://fem-on-colab.github.io/releases/firedrake-install-release-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4767 (4.7K) [application/x-sh]
Saving to: ‘/tmp/firedrake-install.sh’

/tmp/firedrake-inst 100%[===================>]   4.66K  --.-KB/s    in 0s      

2025-12-16 16:26:37 (45.9 MB/s) - ‘/tmp/firedrake-install.sh’ saved [4767/4767]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FIREDRAKE_INSTALLED=/usr/local/share/fem-on-colab/firedrake.installed
+ [[ ! -f /usr/local/share/fem-on-colab/firedrake.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/1f62

### Other

In [4]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

### Burgers

In [7]:
def burgers(h=2**-10, degree=0, nu=0, timestep=2**-5, end_time=1.0, gamma=2**-2):
    mesh = UnitIntervalMesh(round(1/h))
    n = FacetNormal(mesh)
    x, = SpatialCoordinate(mesh)

    V = FunctionSpace(mesh, "DG", degree)
    u_ = Function(V, name="VelocityOld")
    u = Function(V, name="Velocity")
    v = TestFunction(V)

    ic = project((
        conditional(le(abs(x - 0.25), 0.125), 1, 0)
      + conditional(le(abs(x - 0.75), 0.125), - 0.5, 0)
    ), V)

    u_.assign(ic)
    u.assign(ic)

    jump = lambda u: -2 * avg(u * n[0])

    F = (
        inner((u - u_)/timestep, v) * dx
        + 2/3 * inner(avg(u) * jump(u), avg(v)) * dS
        + 1/3 * inner(avg(u * v), jump(u)) * dS
        + gamma * inner(jump(u), jump(v)) * dS
    )
    if degree != 0:
        F += inner(u * u.dx(0), v) * dx
    if nu != 0:
        F += nu * inner(u.dx(0), v.dx(0)) * dx
    if degree != 0 and nu != 0:
        F += (
            inner(avg(u.dx(0)), jump(v))
          + inner(jump(u), avg(v.dx(0)))
        ) * dS

    sp = {
        # "snes_monitor": None,
        # "snes_converged_reason": None,
        "snes_max_it": 100,
    }

    fig, ax = plt.subplots(figsize=(10, 6))
    state = {'t': 0.0}

    def update(frame):
        if frame > 0:
            state['t'] += timestep
            print(GREEN % f"Solving for time t = {state['t']:.4f}:")
            solve(F == 0, u, solver_parameters=sp)
            u_.assign(u)

        ax.clear()
        plot(u, axes=ax, linewidth=3)
        ax.set_title(f"Burgers equation (t = {state['t']:.2f})")
        ax.set_xlabel("x")
        ax.set_ylabel("u")
        ax.set_ylim(-1, 1.5)
        ax.grid(True)

    num_frames = int(end_time / timestep) + 1
    anim = FuncAnimation(fig, update, frames=num_frames, interval=100)
    plt.close()
    return HTML(anim.to_jshtml())

---

## **2. THE IDEA**

How can we meaningfully discuss numerical solutions to ideal systems, like Euler, when we're left with a dichotomy:
- **very weak solutions**, which can have arbitrarily pathological energy behaviour and exhibit a continua of solutions, or
- **weak solutions**, which don't necessarily exist for general boundary conditions.

***Viscosity solutions***

So...
I discussed this issue a bunch with some PDE friends, and one thing that was brought to my attention was *viscosity solutions*.
Letting $(u_\nu)$ be a sequence of weak solutions to a *non*-ideal system with viscosity/dissipation $\nu$ as $\nu \to 0_+$, a very physically meaningful set of solutions to the ideal system is given by those $u$ that are accumulation points of $(u_\nu)$.

If my understanding's correct, these viscosity solutions $u$ are (in general) *very weak* solutions to the ideal system, so we might expect some anomalous energy dissipation.
But this makes a lot of sense!
Each weak solution $(u_\nu)$ will have some energy dissipation, so we can reasonably expect $u$ have it too.
In fact, anomalous dissipation is a very meaningful thing to expect in very weak solutions;
it's like an artifact of the neglected high-order terms that is *in general* negligible, but still manages to contribute to the dynamics through the roughness of in your data.

Viscosity solutions then give us a way to restrict to the physically meaningful set of very weak solutions.
It's fair to say, they struck me as such a wonderfully well-formed way to analyse ideal systems, so I was curious how we might transfer the ideas to the discrete level.

To fix ideas, I'm going to use some examples...

### ***2.1. Example (Burgers)***

Let's look at Burgers' equation,
$$
\dot{u}  =  - u u_x + \nu u_{xx}.
$$

We'd like to consider the ideal limit, $\nu \to 0_+$.
A simple dimensional analysis tells us that, as $\nu \to 0_+$, the length scale for variations in $u$ is $\sim \nu$.

Now, a typical weak formulation for Burgers would be:
> Find $u \in V$ such that
> $$
> \int_\Omega\dot{u}v = - \int_\Omega u u_x v + \nu u_x v_x
> $$
> for all $v \in V$.

Discretely, we could take some finite-dimensional continuous space $V^h$, and define:
> Find $u \in V^h$ such that
> $$
> \int_\Omega\dot{u}v = - \int_\Omega u u_x v + \nu u_x v_x
> $$
> for all $v \in V^h$.

**The problem:**
As $\nu \to 0_+$, the continuous solution wants to exhibit sharp gradients in $u$ over $\sim \nu$ length scales, which can't be resolved inside $V^h$.

**The solution:**
Use a non-conforming method;
they can resolve arbitrarily steep gradients after all.
In particular, let's consider a DG method with a symmetric interior penalty.
Denote the vertices between elements as $\mathcal{V}^h$, with jumps $[\![\cdot]\!]$ and averages $\{\!\!\{\cdot\}\!\!\}$:

> Find $u \in V^h$ such that
> \begin{multline*}
> \int_\Omega\dot{u}v = - \int_\Omega u u_x v - \sum_{\mathcal{V}^h}\left(\frac{2}{3}\{\!\!\{u\}\!\!\}\{\!\!\{v\}\!\!\} + \frac{1}{3}\{\!\!\{uv\}\!\!\}\!\right)\![\![u]\!]  \\
> - \nu \int_\Omega u_x v_x - \nu\sum_{\mathcal{V}^h}\left(\frac{\sigma}{h}[\![u]\!][\![v]\!] + \{\!\!\{u_x\!\}\!\!\}[\![v]\!] + [\![u]\!]\{\!\!\{v_x\!\}\!\!\}\right)
> \end{multline*}
> for all $v \in V^h$.

That funny form of the convective term is just there to ensure skew-symmetry.

Considering $\nu \to 0_+$, there's two equivalent ways to proceed here, with the same outcome:

---

***Idea 1***

One way to derive a non-conforming discretisation is via mollification.
Take a mollification operator $\mathcal{M}_{h/\sigma}$, which mollifies over a distance $h/\sigma$.
We consider our conforming weak form, then instead of taking $u = u^h$, $v = v^h$ for conforming $u^h$, $v^h$, we take $u = \mathcal{M}_{h/\sigma}[u^h]$, $v = \mathcal{M}_{h/\sigma}[v^h]$ for non-conforming $u^h$, $v^h$.
One then discards all terms that are $o[1]$ in $\sigma$, and gets a non-conforming disretisation.
Easy peasy.
If we fancied making our lives difficult for ourselves, we could have constructed the above this way.

From our dimensional analysis on the continuous level, we know we should expect gradients over a length scale $\sim \nu$.
So to accurately reproduce the dynamics of the continuous level, it makes sense to mollify over lenth scale $\sim \nu$.
So let's take $h / \sigma \sim \nu$.

***Idea 2***

We'd like to take $\nu \to 0_+$.
But if we do this without scaling $\sigma$, then the viscous term will vanish entirely, which is (i) not appropriate for viscosity solutions, and (ii) will make the problem lose its well-posedness.
So how do we scale $\sigma$ with $\nu$?

We'd like the leading vertex contributions of $\int_\Omega u u_x v$ and $\nu \int_\Omega u_x v_x$ to balance as $\nu \to 0_+$.
This necessitates $h / \sigma \sim \nu$.

---

Taking $h / \sigma = \nu / \gamma$ and pushing $\nu \to 0_+$, we're left with the following scheme for viscosity solutions of the ideal form of Burgers' equation:

> Find $u \in V^h$ such that
> \begin{multline*}
> \int_\Omega\dot{u}v = - \int_\Omega u u_x v - \sum_{\mathcal{V}^h}\left(\frac{2}{3}\{\!\!\{u\}\!\!\}\{\!\!\{v\}\!\!\} + \frac{1}{3}\{\!\!\{uv\}\!\!\}\!\right)\![\![u]\!]  \\
> - \gamma\sum_{\mathcal{V}^h}[\![u]\!][\![v]\!]
> \end{multline*}
> for all $v \in V^h$.

It makes sense to ensure $u \to \gamma \sum_{\mathcal{V}^h}[\![u]\!]^2$ defines a norm on $V^h$, which is the case when $V^h$ is DG0.

In [9]:
burgers()

Solving for time t = 0.0312:
Solving for time t = 0.0625:
Solving for time t = 0.0938:
Solving for time t = 0.1250:
Solving for time t = 0.1562:
Solving for time t = 0.1875:
Solving for time t = 0.2188:
Solving for time t = 0.2500:
Solving for time t = 0.2812:
Solving for time t = 0.3125:
Solving for time t = 0.3438:
Solving for time t = 0.3750:
Solving for time t = 0.4062:
Solving for time t = 0.4375:
Solving for time t = 0.4688:
Solving for time t = 0.5000:
Solving for time t = 0.5312:
Solving for time t = 0.5625:
Solving for time t = 0.5938:
Solving for time t = 0.6250:
Solving for time t = 0.6562:
Solving for time t = 0.6875:
Solving for time t = 0.7188:
Solving for time t = 0.7500:
Solving for time t = 0.7812:
Solving for time t = 0.8125:
Solving for time t = 0.8438:
Solving for time t = 0.8750:
Solving for time t = 0.9062:
Solving for time t = 0.9375:
Solving for time t = 0.9688:
Solving for time t = 1.0000:


Viscosity solutions to Burgers' should have shocks that satisfy the Rankine–Hugoniot condition,
$$
u_{\text{shock}} = \frac{1}{2}u_+ + \frac{1}{2}u_-.
$$
We see these indeed that these are satisfied by our numerical solutions!

[Note about how this looks like CG1]

### ***2.2. Example (Navier–Stokes/Euler)***

If we can do Burgers, we can do Navier–Stokes:
$$
\dot{\mathbf{u}}  =  - \, \mathbf{u} \cdot \nabla\mathbf{u} - \nabla p + \frac{2}{\text{Re}}\rm{div} \, \nabla_\rm{s} \mathbf{u},  \qquad
0  =  \rm{div}\,\mathbf{u}.
$$

A typical weak formulation would be:
> Find $(\mathbf{u}, p) \in V \times Q$ such that
>
> \begin{align*}
> \int_\Omega\dot{\mathbf{u}}\cdot\mathbf{v}  &=  \frac{1}{2} \int_\Omega [(\mathbf{u}\cdot\nabla\mathbf{v})\cdot\mathbf{u} - (\mathbf{u}\cdot\nabla\mathbf{u})\cdot\mathbf{v}] + \int_\Omega p \, (\rm{div}\,\mathbf{u}) \\
> &\qquad\qquad\qquad\qquad\qquad\qquad\qquad+ \frac{2}{\rm{Re}}\int_\Omega \nabla_\rm{s}\mathbf{u} : \nabla_\rm{s}\mathbf{v},  \\
> 0  &=  \int_\Omega (\rm{div}\,\mathbf{u}) \, q,
> \end{align*}
>
> for all $(\mathbf{v}, q) \in V \times Q$.

[Note on gradient jump penalisation.]

### ***2.3. Example (Allen–Cahn)***

### ***2.4. Example (implicit Euler)***